<a href="https://colab.research.google.com/github/anujoshi3390/Wine-Recomendations/blob/main/Wine_Recommendations_EAD%2C_Models%2C_Explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Understanding
The dataset was scraped from Vivino.com. It contains mostly textual data, but some of the data is numeric. Each row is specific to a bottle of wine. It contains the following information:

-winery: str, the winery that makes that specific wine
-wine_name: str, the name of the wine
-wine_type: str, the classification of the wine (ie red, white)
-wine_country: str, the country in which the wine was made
-average_rating: float, the average rating of the wine on Vivino.com
-num_of_ratings: int, the number of ratings for that wine on Vivino.com
-wine_price: float, the price per bottle
-grapes: str, the grapes utilized in the wine
-alcohol_content: float, the percent alcohol in the wines
-reviews: str, top 3 most helpful text reviews of the wine

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

In [3]:
#bringing in the dataframe
df = pd.read_csv('wine_info.csv')

In [4]:
#checking the first 5 rows of the DataFrame
df.head()

,winery,wine_name,wine_type,wine_country,average_rating,num_of_ratings,wine_price,grapes,alcohol_content,reviews
0,El Enemigo,Chardonnay 2018,White wine,Argentina,4.3,2239,23.99,100% Chardonnay,13.8,"4.25 perhaps. Apple, then more Chardonnay like..."
1,Minuty,M Rosé 2020,Rosé wine,France,4.0,354,18.99,"50% Grenache, 40% Cinsault, 10% Shiraz/Syrah",12.5,"3,8. Pale salmon color. Nose of peach, strawbe..."
2,Santi,Infinito Rosé 2018,Rosé wine,Italy,3.8,93,12.29,"65% Corvina, 30% Rondinella, 5% Molinara",12.0,"Dejlig, let neutral rosé. God til pizza ved ga..."
3,Jansz,Premium Rosé N.V.,Sparkling wine,Australia,3.9,1460,29.00,"Pinot Meunier, Pinot Noir, Chardonnay",12.0,89/100 Blend of 78/22/ Pinot Noir and Chardonn...
4,Balbach,Riesling 2018,White wine,Germany,4.3,49,14.99,Riesling,NaN,extraordinary balance between acidity and swee...


In [6]:
#This is the current size of the dataframe
df.shape


(1800, 10)

In [5]:
#checking for duplicates
sum(df.duplicated())

11

In [7]:
df.drop_duplicates(inplace = True)

In [9]:
df.dtypes

,0
winery,object
wine_name,object
wine_type,object
wine_country,object
average_rating,float64
num_of_ratings,int64
wine_price,float64
grapes,object
alcohol_content,float64
reviews,object


In [ ]:
df.describe()

Data Cleaning



In [ ]:
#combining the winery and wine_name to get the full name of the product, the combination named 'title', and then
#assigning it to the index
df['title'] = df['winery'].str.cat(df['wine_name'], sep = ' ')
df.set_index('title', inplace = True)

In [ ]:
#creating a function to remove the year from the 'wine_name' column
def remove_year_in_name(x):
    '''
    This function removes the year from the
    wine (name of wine) column.
    '''
    #These are the typical years of wines on Vivino.com
    if '17' or '18' or '19' or '20' in x:
        x = x[:-5]
        return x
    else:
        return x

I have decided to remove the year from the 'wine_name' column for easier filtering. The year will still be a part of the full 'title' variable that is now the index of the dataset.

In [ ]:
#applying the function above
df['wine_name'] = df['wine_name'].apply(remove_year_in_name)

In [ ]:
df.isnull().sum()

This verifies that a good amount of the entries for alcohol_content are not available. Perhaps the column should be dropped. Wine price, I believe is something that can be imputed, as for the reviews, I would not suggest any filling techniques for the null values.

In [ ]:
#dropping the alcohol column
df.drop('alcohol_content', axis = 1, inplace = True)

With some of the price variables missing, I am going to impute them with the mean

In [ ]:
#imputing the mean
wine_mean = df.wine_price.mean()

df['wine_price'].fillna(wine_mean, inplace = True)

Below I will remove the null review entries.

In [ ]:
Below I will remove the null review entries.

In [ ]:
#dropping the rows with the missing reviews
df.dropna(subset = ['reviews'], inplace = True)

In [ ]:
#creating a function to fix a capitalization error
def lowercase_W(x):
    """
    This function changes the capitalization error
    in Fortified Wine
    """
    if x == 'Fortified Wine':
        x = 'Fortified wine'
        return x
    else:
        return x

In [ ]:
df['wine_type'] = df['wine_type'].apply(lowercase_W)

In [ ]:
#changing the wine_type to a single word
conditions = [df['wine_type'] == 'White wine',df['wine_type'] == 'Red wine',
               df['wine_type'] =='Sparkling wine',df['wine_type'] == 'Rosé wine',
               df['wine_type'] =='Dessert wine', df['wine_type'] =='Fortified wine']
choices = ['White', 'Red', 'Sparkling', 'Rosé', 'Dessert', 'Fortified']

df['wine_type'] = np.select(conditions, choices)

In [ ]:
#creating a function to clean the grape column
def clean_grape_column(x):
    '''
    This function cleans the entries in the grape
    column to make all grapes have
    only 1 name. It removes the '100 %', and it
    also makes any 2 or more grapes in 1 wine
    as a blend.
    '''
    if type(x) == float: #this will keep the NaN values in tact
        return x
    if ',' in x:
        return 'Blend'
    elif '%' in x:
        return ' '.join(x.split()[1:])
    else:
        return x


In [ ]:
df['grapes'] = df['grapes'].apply(clean_grape_column)

In [ ]:
df[df['grapes'].isna()].head(2)

I was hoping that for these NaN values that the name of the grape would be in the wine_name column as they typically would be a part of the name of the wine. Because they are not for these remaining N/A grapes, I will have to drop these rows.

In [ ]:
df.dropna(subset=['grapes'], inplace = True)

Now that I have categorized 2 or more grapes as a blend, I can get a little more specific in terms of the type of blend based on the type of wine:

In [ ]:
#changing the Blend to Red, White Blend etc...
types = df['wine_type'].tolist()
grapes = df['grapes'].tolist()

for i, n in enumerate(grapes):
    if grapes[i] == 'Blend':
        grapes[i] = types[i] + ' ' + 'Blend'

df['grapes'] = grapes

In [ ]:
df.shape

This dataset now has 1770 wines.

## **Exploratory Data Analysis**

In [ ]:
#looking at unique wineries
df['winery'].nunique()

In [ ]:
#looking at the wines by winery
df.groupby('winery')['wine_name'].count().sort_values(ascending = False).head()

In [ ]:
#creating a chart of wines by winery
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(header=dict(values=['Winery', 'Number of Wines']),
                 cells=dict(values=[['Lustau', 'Quady', 'Sandeman', 'Taylor\'s', 'Warre\'s'], [16, 14, 13, 11, 10]]))
                     ])
fig.update_layout(
    autosize=False,
    width=500,
    height=400,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=1
    ),
)
fig.show()

In [ ]:
#creating a graph of wines by winery
labels = list(df.groupby('winery')['wine_name'].count().sort_values(ascending = False).head(5).index)

fig, ax = plt.subplots()
sns.barplot(x = df.groupby('winery')['wine_name'].count().sort_values(ascending = False).head(5).index, y = df.groupby('winery')['wine_name'].count().sort_values(ascending = False).head(5).values)
ax.set_title('Wines by Winery (Top 5 Representation)')
ax.set_xlabel('Winery Name')
ax.set_ylabel('Frequency')
ax.set_xticklabels(labels, rotation = 45)
plt.savefig('Images/wine_representation.png', facecolor='white' );


In [ ]:
#creating a graph of wines by winery
labels = list(df.groupby('winery')['wine_name'].count().sort_values(ascending = False).head(5).index)

fig, ax = plt.subplots()
sns.barplot(x = df.groupby('winery')['wine_name'].count().sort_values(ascending = False).head(5).index, y = df.groupby('winery')['wine_name'].count().sort_values(ascending = False).head(5).values)
ax.set_title('Wines by Winery (Top 5 Representation)')
ax.set_xlabel('Winery Name')
ax.set_ylabel('Frequency')
ax.set_xticklabels(labels, rotation = 45)
plt.savefig('Images/wine_representation.png', facecolor='white' );

It seems the Lustau winery has 16 different products represented in this dataset. There are a few other wineries with various products in this dataset. I will hopefully use that later on and create a version of a recommender based on winery.



In [ ]:
#looking at the distributon of wine types
df['wine_type'].value_counts()

In [ ]:
#graphing the distribution of wines by type and country
fig, axes = plt.subplots(1, 2, figsize=(17, 5))

one = sns.barplot(ax=axes[0], x=df['wine_type'].value_counts().index, y=df['wine_type'].value_counts().values)
axes[0].set_title('Wines by Type')
axes[0].set_xlabel('Type of Wine')
axes[0].set_ylabel('Frequency')

two = sns.barplot(ax=axes[1], x=df['wine_country'].value_counts().index, y=df['wine_country'].value_counts().values)
two.set_xticklabels(df['wine_country'].value_counts().index, rotation = 45)
axes[1].set_title('Wines by Country')
axes[1].set_xlabel('Country')
axes[1].set_ylabel('Frequency of Wines')

plt.tight_layout()
plt.savefig('Images/wine_graphs.png', facecolor='white' );

There seems to be a fairly even distribution of wines in the dataset. The majority of the wines in this dataset come from France, the United States and Italy. I did not find this surprising at all.

In [ ]:
#graphing the ratings data
fig, ax = plt.subplots()
sns.barplot(df['average_rating'], df['num_of_ratings'])
ax.set_xlabel('Average Rating of Wines')
ax.set_ylabel('Number of Wines with Average Rating')
ax.set_title('Number of Ratings vs Average Ratings');


The ratings of wines are from 1 - 5. The range of ratings in this dataset is moreso in the middle. There is no rating less than 2.6 and none larger than 4.6. This means most of the wines have a fairly similar rating. The mode of the ratings collected is the 3.8 mark. This could be representative of truly average thoughts about the wine.



In [ ]:
df.grapes.value_counts().head()

In [ ]:
#graphing the number of wines per grape
fig, ax = plt.subplots(figsize=(20,6))
sns.barplot(df['grapes'].value_counts().index, df['grapes'].value_counts().values)
ax.set_xlabel('Types of Grapes')
ax.set_ylabel('Number of Wines')
ax.set_xticklabels(df['grapes'].value_counts().index, rotation = 90)
ax.set_title('Number of Wines Per Grape');


Most of the wines in this dataset are Fortified blends. Infact, I would say that blends make up the majority of wines in the dataset. This shows that the combination of grapes in a wine is a popular technique.



In [ ]:
#graphing a histogram of wine prices
fig, ax = plt.subplots()
sns.distplot(df['wine_price'], kde = False)
ax.set_title('Histogram of Wine Prices')
ax.set_xlabel('Price in Dollars')
ax.set_ylabel('Frequency of Wines');

The majority of the wines prices seem to fall around the 25-30 USD mark. The range of the wines is quite large, giving room for suggestion to create a recommender based on price as another variation of a rec system.

In [ ]:
df.groupby('wine_type')['wine_price'].mean().index

In [ ]:
#graphing the price by wine type
fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(df.groupby('wine_type')['wine_price'].mean().index, df.groupby('wine_type')['wine_price'].mean().values)
ax.set_xlabel('Wine Type')
ax.set_ylabel('Wine Price Mean (in USD)')
ax.set_title('Price by Wine Type');


It seems that red wine tends to be a bit pricer than others, however, in the scope of things, the difference in average price between the types of wine is not much. The range in price is only 15 USD. This means that if a person were to favor a specific wine type it would not cost them too much more for their preference.



## Data Preprocessing

**Natural Language Processing**



Now that I understand the data better, I can move on to preprocessing the text reviews using NLP. In this step, I will be adding the grapes and wine types into the stopwords as well so that I can perform a Classification algorithm on them below.



In [ ]:
#importing NLP libraries
from nltk.corpus import stopwords
from nltk import pos_tag
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
#generating the list of NLTK's typical english stop words
sw = stopwords.words('english')

#this is a list of grapes lowercased to add to stopwords
wine_string = ','.join(list(df['grapes'].str.lower().unique()))
if ' ' in wine_string:
    wine_string = wine_string.replace(' ', ',')

grape_sw = wine_string.split(',')

#compiling a list of wine_types to add to stop words as well
kind_sw = []

for kind in list(df['wine_type'].unique()):
    kind = kind.lower().replace(' wine', '')
    kind_sw.append(kind)


#adding the grape & kind stop words to the list of all stop words
sw.extend(grape_sw)
sw.extend(kind_sw)

#adding additional words in as well that are unnecessary:
unnecessary_words = ['port', 'good', 'nice', 'smell', 'taste', 'mouth',
                     'nose', 'great', 'note', 'finish', 'tannin',
                     'wine', 'like', 'palate', 'aroma', 'one', 'hint',
                    'well', 'long', 'short', 'ro', 'ros', 'perfect',
                     'flavor', 'lovely', 'really', 'very', 'bit', 'little',
                     'make', 'open', 'lot', 'go', 'big',
                    'age', 'brioche', 'bottle', 'drink', 'toast']
sw.extend(unnecessary_words)

In [ ]:
def get_wordnet_pos(treebank_tag):
    '''
    This function will convert nltk POS
    to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


In [ ]:
def text_preparer(doc, stopwords=sw):
    '''
    This function takes in a document, and a list of
    stop words. It prepares the text by removing the numbers
    and punctuation (with Regex), lowercasing the
    elements, and lemmatizing the text.
    '''

    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in sw]
    doc = pos_tag(doc)
    #The lemmatizer wants to know what to look for:
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer()
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)

In [ ]:
#parsing the review text
df['token_reviews'] = df['reviews'].apply(text_preparer)

In [ ]:
#getting the groups together to make word clouds
White = df[df['wine_type'] == 'White']['token_reviews'].values.tolist()
Red = df[df['wine_type'] == 'Red']['token_reviews'].values.tolist()
Sparkling = df[df['wine_type'] == 'Sparkling']['token_reviews'].values.tolist()
Fortified = df[df['wine_type'] == 'Fortified']['token_reviews'].values.tolist()
Dessert = df[df['wine_type'] == 'Dessert']['token_reviews'].values.tolist()
Rosé = df[df['wine_type'] == 'Rosé']['token_reviews'].values.tolist()
texts = [White, Red, Sparkling, Fortified, Dessert, Rosé]

#these will be the labels for word clouds
labels = ['White', 'Red', 'Sparkling', 'Fortified', 'Dessert', 'Rosé']

In [ ]:
from wordcloud import WordCloud

def show_wordcloud(data, labels):
    """
    This function takes a list of text data variables
    (the data itself must also being in list form)
    and it also takes a list of labels and returns
    a word cloud.
    """
    i = 0
    for group in data:
        wordcloud = WordCloud(
            background_color='black',
            stopwords=sw,
            max_words=30,
            scale=3,
            random_state=12
        ).generate(str(group))

        fig = plt.figure(1, figsize=(6, 6))
        plt.axis('off')
        plt.title(labels[i])
        plt.imshow(wordcloud)
        plt.tight_layout(pad=0)
        #This saves each figure individually!
        plt.savefig(f'Images/{labels[i]}Cloud.png', transparent = True)
        plt.show()
        i+=1

In [ ]:
#generating word clouds for each of the wine types
show_wordcloud(texts, labels)

With some domain knowledge of the conventional descriptions of these wine types, I would say that these reviews seem to do a good job of characterizing their labels.

## Vectorization & Standardization

Before I do some modeling, I will use TFIDF to vectorize my text reviews, merge the array to the DataFrame, and scale:

In [ ]:
#vectorizing text reviews
from sklearn.feature_extraction.text import TfidfVectorizer

#setting the parameters to be able to filter for top words
tfidfvec = TfidfVectorizer(sublinear_tf=True, max_df=.9, min_df=.05,  ngram_range=(1, 1))
tf_idfwine = tfidfvec.fit_transform((df_model['token_reviews']))


In [ ]:
#using feature_names to see the top words
df_tfidf = pd.DataFrame(tf_idfwine.toarray(), columns = tfidfvec.get_feature_names())

In [ ]:
df_tfidf.head(2)

In [ ]:
#dropping unnecessary columns from the df_model DataFrame
model_data = df_model.drop(labels = ['winery', 'wine_name', 'wine_type', 'reviews', 'token_reviews'], axis = 1)
model_data = pd.concat([model_data.reset_index(), df_tfidf], axis = 1)
model_data.set_index('title', inplace = True)

In [ ]:
model_data.columns[:3] #aside from the dummy variables and vectorized columns, these are the other ones in the model

Index(['average_rating', 'num_of_ratings', 'wine_price'], dtype='object')


In [10]:
#scaling the new matrix
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(model_data)

scale_model_data = pd.DataFrame(scaler.transform(model_data), columns = model_data.columns, index = model_data.index)


NameError: name 'model_data' is not defined

### **Modeling**

**Classification**

As a form of supervised learning, and a clarity check, I am going to perform some Classification algorithms on the text reviews and assess their accuracy on identifying the type of wine in the 'wine_types' column.



**Random Forest**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df['reviews']
y = df['wine_type']


In [ ]:
#splitting the data
X_train, X_test, y_train, y_test =  train_test_split(X, y, train_size= .8, random_state= 13)

In [ ]:
#creating a RandomForest pipeline with a grid search
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
accuracy = make_scorer(accuracy_score)

pipeline = Pipeline([
                    ('tv', TfidfVectorizer()),
                    ('rf', RandomForestClassifier(random_state=42))
                    ])


rf_param_grid_pipe = {
    "rf__criterion": ["gini", "entropy"],
    "rf__max_depth": [None, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16],
    "rf__max_features": [None,4,5,6,9,10],
    "rf__min_samples_split": [2, 5, 10],
    "rf__min_samples_leaf" : [1, 2, 3, 5, 6],
    "rf__n_estimators" : [10, 30, 100]
}


grid_search_pipe = GridSearchCV(pipeline, rf_param_grid_pipe, cv=3, verbose=1, n_jobs=-1, scoring = accuracy)

grid_search_pipe.fit(X_train, y_train)


In [ ]:
print(grid_search_pipe.best_params_)

In [ ]:
grid_search_pipe.score(X_test, y_test)

0.7768361581920904
78% accuracy is not too terrible. I would be curious to see if a MultiNomial Naive Bayes model would have better accuracy performance.

## **MultiNomial Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mb = Pipeline([
                    ('tv', TfidfVectorizer()),
                    ('mb', MultinomialNB())
                    ])

param_grid_pipe = {
    "mb__alpha": [1, .5, .25, .75],

}


grid_search_pipe = GridSearchCV(mb, param_grid_pipe, cv=3, verbose=1, n_jobs=-1, scoring=accuracy)
grid_search_pipe.fit(X_train, y_train)

In [ ]:
print(grid_search_pipe.best_params_)

In [ ]:
grid_search_pipe.score(X_test, y_test)

0.8333333333333334
When comparing the two algorithms, the Multinomial Bayes gave a higher accuracy score. I will now run a model with those parameters to look at the confusion matrix.



In [ ]:
mb = Pipeline([
                    ('tv', TfidfVectorizer()),
                    ('mb', MultinomialNB(alpha = .25))
                    ])

mb.fit(X_train, y_train)

Pipeline(steps=[('tv', TfidfVectorizer()), ('mb', MultinomialNB(alpha=0.25))])


In [ ]:
y_pred_mb = mb.predict(X_test)

In [ ]:
from sklearn.metrics import plot_confusion_matrix, classification_report
plot_confusion_matrix(mb,y_pred_mb, y_test)
plt.savefig('Images/Matrix.png');

The sparkling and dessert wine categories seemed to perform the worst out of all them. I believe this is due to the broadness of the categories themselves. The other categories seem to be a little more specifically defined.



In [ ]:
from sklearn.metrics import plot_confusion_matrix, classification_report
print(classification_report(y_test, y_pred_mb))

The reviews show an 83% accuracy of identifying the types of wine. This shows that the reviews can be helpful in making recommendations.

# **Recommender Systems**

By Title (Wine Full Name)

Cosine Similarity Model

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#calculating the cosine similarities of the model on itself.
cos_sim = cosine_similarity(scale_model_data, scale_model_data)

In [ ]:
# Storing indices of the data
indices = pd.Series(df.index)

def cosine_recommendations(title, cos_sim, num_recommendations):
    """
    This function takes in a title from the index along
    with the cosine similarity array, and number of recommendations.
    It returns the top wines recommended with the highest similarity
    scores.
    """
    recommended_wines = []
    index = indices[indices == title].index[0]
    similarity_scores = pd.Series(cos_sim[index]).sort_values(ascending = False)
    top_recommended_wines = list(similarity_scores.iloc[1:num_recommendations+1].index)
    for i in top_recommended_wines:
        recommended_wines.append(list(df_model.index)[i])
        matches = df.loc[recommended_wines][['wine_type', 'wine_country', 'grapes', 'average_rating', 'wine_price']]
    matches['cosine_similarity_scores'] = list(similarity_scores.iloc[1:num_recommendations+1].values)
    return matches

In [ ]:
print('Cosine Similarity Model by Title \n1000 Stories Chardonnay 2018')
stories_cosine = cosine_recommendations("1000 Stories Chardonnay 2018", cos_sim, 10)
stories_cosine

Cosine Similarity Model by Title
1000 Stories Chardonnay 2018


In [ ]:
df.loc['1000 Stories Chardonnay 2018'][['wine_type', 'wine_country', 'grapes', 'average_rating', 'wine_price']]

Using Cosine Similarity as a metric, these are the top 10 wines that are similar to the '1000 Stories Chardonnay 2018' wine I have set as a parameter to the function. The majority of the other wines recommended are also US Chardonnays.


This time, using Euclidean Distance as a metric, these are the top 10 wines recommended for that same bottle. Again, it seems logical so far because most of these wines are US Chardonnays like the original one in the input.

In order to compare the two methods together, I think I will have to check the wine_price and wine_rating variables. I am hoping this will probably give a better sense of which method yielded the best results.


In [ ]:
#creating a dataframe to get a closer look at the other variables.
comparison = pd.DataFrame({'1000 Stories Chardonnay 2018': [df.loc['1000 Stories Chardonnay 2018']['average_rating'], df.loc['1000 Stories Chardonnay 2018']['wine_price']],
             'Cosine Model': [stories_cosine['average_rating'].mean(), stories_cosine['wine_price'].mean()],
             'Euclidean Model': [stories_euclidean['average_rating'].mean(), stories_euclidean['wine_price'].mean()]}, index = ['Average Rating', 'Wine Price'])


Taking a closer look at the other two variables, rating and price, it is a close call. The Euclidean Model seems to be a little bit closer to the average rating and wine price of the Stories Chardonnay bottle. This is based on it's overall average of those columns.

# By Winery

Cosine Similarity


In this dataset, there were 16 wines produced by a single winery. I would be curious to see these recommendations.



In [ ]:
df[df['winery'] == 'Lustau'].head(2)

In [ ]:
#creating a dataframe of the wines from the Lustau winery in Spain
lustau = df[df['winery'] == 'Lustau']

print('Cosine Similarity Model by Winery')
lustau_cosine = cosine_recommendations('Lustau San Emilio Pedro Ximenez N.V.', cos_sim, 10)
lustau_cosine

These are the top 10 wines recommended from the Lustau winery after choosing the Lustau San Emilio Pedro Ximenez N.V. as a base for recommendations.



**Euclidean Distance**

These are the top 10 recommendations from the Lustau winery using Euclidean Distance.

In comparison here, the two models have output the same exact wines, however the order of the wines differs between models. Due to the small inventory of 16 wines to choose from, it is highly likely that the algorithms would output the same products.

In [ ]:
#creating a dataframe to get a closer look at the other variables.
pd.DataFrame({'Lustau San Emilio Pedro Ximenez N.V.': [df.loc['Lustau San Emilio Pedro Ximenez N.V.']['average_rating'], df.loc['Lustau San Emilio Pedro Ximenez N.V.']['wine_price']],
             'Cosine Model': [lustau_cosine['average_rating'].mean(), lustau_cosine['wine_price'].mean()],
             'Euclidean Model': [lustau_euclidean['average_rating'].mean(), lustau_euclidean['wine_price'].mean()]}, index = ['Average Rating', 'Wine Price'])


Taking a closer look here, the models have identical average scores due to them outputting the same wines. To evaluate this it would come down to more of an opinion. I think that for a presentation, it would be more convincing to show the best results from the By Title recommender at this point in time, I will have to revisit the By Winery recommender in the near future.

In [ ]:
#saving the data for Streamlit app.
df.to_csv('Streamlit/df.csv')
scale_model_data.to_csv('Streamlit/scale_model_data.csv')

Evaluation
The results of my content based recommendation systems, created using the cosine similarity and euclidean distancing metrics, are difficult to interpret by the numbers as they are fairly similar when comparing the average price and average rating. For the By Title model, the numbers seemed to favor the Euclidean Distance model. For the By Winery model, I would say that the best results may have to be based on opinion. All variations of the models deep down are subject to opinion anyway because of the nature of a content based recommender. On that note of opinion, I had a couple of my friends, who are avid wine drinkers, test the models and they did feel that the output of the Euclidean Distance metric on all models seemed a bit more logical based on the order of their recommendations. I am assured on the recommendation system as a whole, having performed classification algorithms on the review text, and achieving the accuracy results I did.